<a href="https://colab.research.google.com/github/moizmoizmoiz/nlp-cw/blob/main/nlp_cw(beta3.1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install tensorflow_datasets 
%pip install tf-models-official
%pip install transformers
%pip install tensorflow
%pip install https://github.com/scikit-learn/scikit-learn/archive/master.zip
# Installing additional libraries for text preprocessing
%pip install emoji
%pip install contractions
%pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 8.3 MB 9.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_loggin

In [2]:
import sys, sklearn, os, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt
from functools import partial
import PIL, PIL.Image, pandas as pd
import tensorflow as tf, tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers, initializers
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support
from transformers import RobertaConfig, BertTokenizer, AutoTokenizer, TFAutoModel, AutoModel, AdamW, InputExample, InputFeatures, BertTokenizerFast
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.optimizers import Adam
import collections, re, unicodedata, emoji, contractions

In [17]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import pandas as pd

# %tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support

from tensorflow.keras.initializers import TruncatedNormal
from keras import backend as K

from transformers import RobertaConfig
from transformers import BertTokenizer, AutoTokenizer, TFAutoModel, AutoModel, AdamW, InputExample, InputFeatures, TFBertModel, BertTokenizerFast
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.optimizers import Adam

import collections
import re
import unicodedata
import emoji
import contractions

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

In [16]:
ds = tfds.load('goemotions')
#look up how it can split it splits easily

In [30]:
def remove_multilabels(ds, split):
    df = tfds.as_dataframe(ds[split])
    df = df[df[df.columns.difference(['comment_text'])].sum(axis=1) == 1]
    return df

In [36]:
def merge_columns(df, split):
    df.loc[:, 'love'] += df['caring']
    df.loc[:, 'approval'] += df['admiration']
    df.loc[:, 'disapproval'] += df['disgust'] + df['anger'] + df['annoyance']
    df.loc[:, 'surprise'] += df['curiosity']
    df.loc[:, 'fear'] += df['nervousness']
    df.loc[:, 'sadness'] += df['embarrassment'] + df['grief'] + df['remorse'] + df['disappointment']
    df.loc[:, 'joy'] += df['amusement'] + df['excitement'] + df['relief']
    df.drop(['caring', 'admiration', 'disgust', 'anger', 'annoyance', 'curiosity', 'nervousness', 'embarrassment',
              'grief', 'remorse', 'disappointment', 'relief', 'excitement', 'amusement'], axis=1, inplace=True)
    return df


In [35]:
ds_train = merge_columns(remove_multilabels(ds, 'train'), 'train')
ds_valid = merge_columns(remove_multilabels(ds, 'validation'), 'validation')
ds_test = merge_columns(remove_multilabels(ds, 'test'), 'test')

<ipython-input-34-6dfe785b2258>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'love'] += df['caring']
<ipython-input-34-6dfe785b2258>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'approval'] += df['admiration']
<ipython-input-34-6dfe785b2258>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [37]:
print(ds_train.columns)

Index(['approval', 'comment_text', 'confusion', 'desire', 'disapproval',
       'fear', 'gratitude', 'joy', 'love', 'neutral', 'optimism', 'pride',
       'realization', 'sadness', 'surprise'],
      dtype='object')


In [38]:
ds_train.head()

,approval,comment_text,confusion,desire,disapproval,fear,gratitude,joy,love,neutral,optimism,pride,realization,sadness,surprise
0,False,"b""It's just wholesome content, from questionab...",False,False,False,False,False,False,False,True,False,False,False,False,False
1,True,b'This is actually awesome.',False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,b'I grew up on the other side of Ama but live ...,False,False,False,False,False,False,False,True,False,False,False,False,False
5,False,b'I second disagreeing that they are awesome.',False,False,True,False,False,False,False,False,False,False,False,False,False
6,False,b'fucking fuck fuck',False,False,False,False,False,False,False,True,False,False,False,False,False


In [39]:
for ds in [ds_train, ds_valid, ds_test]:
    ds['comment_text'] = ds['comment_text'].str.decode("utf-8")

In [40]:
ds_train.head()

,approval,comment_text,confusion,desire,disapproval,fear,gratitude,joy,love,neutral,optimism,pride,realization,sadness,surprise
0,False,"It's just wholesome content, from questionable...",False,False,False,False,False,False,False,True,False,False,False,False,False
1,True,This is actually awesome.,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,I grew up on the other side of Ama but live in...,False,False,False,False,False,False,False,True,False,False,False,False,False
5,False,I second disagreeing that they are awesome.,False,False,True,False,False,False,False,False,False,False,False,False,False
6,False,fucking fuck fuck,False,False,False,False,False,False,False,True,False,False,False,False,False


In [41]:
GE_taxonomy = ['approval',
               'confusion', 'desire',
               'disapproval', 'fear',
               'gratitude','joy', 'love', 'neutral',
               'optimism', 'pride', 'realization', 'sadness',
               'surprise']

In [42]:
num_labels = len(GE_taxonomy)
print(num_labels)

14


In [45]:
max_length = pd.concat([ds_train['comment_text'], ds_test['comment_text'], ds_valid['comment_text']]).apply(lambda x: len(x.split())).max()

max_length

32

In [47]:
# Building a preprocessing function to clean text
def preprocess_corpus(x):

  # Adding a space between words and punctation
  x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
  x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)
  
  # Demojize
  x = emoji.demojize(x)
  
  # Expand contraction
  x = contractions.fix(x)
  
  # Lower
  x = x.lower()
  # # Handling emojis
  x = re.sub(r"<3", " love_heart ", x)
  x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
  x = re.sub(r":\)", " smiling_face ", x)
  x = re.sub(r"^_^", " smiling_face ", x)
  x = re.sub(r"\*_\*", " star_struck ", x)
  x = re.sub(r":\(", " frowning_face ", x)
  x = re.sub(r":\^\(", " frowning_face ", x)
  x = re.sub(r";\(", " frowning_face ", x)
  x = re.sub(r":\/",  " confused_face", x)
  x = re.sub(r";\)",  " wink", x)
  x = re.sub(r">__<",  " unamused ", x)
  x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
  x = re.sub(r"\b(n+a+h+)\b", "nah", x)

  # Remove special characters and numbers replace by space + remove double space
  x = re.sub(r"\b([.]{3,})"," dots ", x)
  x = re.sub(r"[^A-Za-z!?_]+"," ", x)
  x = re.sub(r"\b([s])\b *","", x)
  x = re.sub(r" +"," ", x)
  x = x.strip()

  return x

In [48]:
# Applying the preprocessing function on the dataset
datasets = [ds_train, ds_valid, ds_test]
for dataset in datasets:
    dataset["clean_text"] = dataset["comment_text"].apply(preprocess_corpus)


# Preview of data
display(ds_train[['comment_text', 'clean_text']].sample(5))

<ipython-input-48-fb4aceb1996c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["clean_text"] = dataset["comment_text"].apply(preprocess_corpus)
<ipython-input-48-fb4aceb1996c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["clean_text"] = dataset["comment_text"].apply(preprocess_corpus)
<ipython-input-48-fb4aceb1996c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,comment_text,clean_text
2104,"Wow your sister is in abusive relationship, co...",wow your sister is in abusive relationship cool
30853,If you're blaming her for the design of the mo...,if you are blaming her for the design of the m...
10854,Being compared to [NAME] is a compliment becau...,being compared to name is a compliment because...
8734,They just dont want to go to school. Source:am...,they just do not want to go to school source a...
5755,I have trouble even remembering what true happ...,i have trouble even remembering what true happ...


In [50]:
from transformers import RobertaConfig
from transformers import RobertaTokenizer
from transformers import TFRobertaModel

In [51]:
model_name = "roberta-base"
configuration = RobertaConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
transformer_model = TFRobertaModel.from_pretrained(model_name, configuration)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [47]:
# function for creating BERT based model
def create_model(nb_labels):
  #nb_labels is the number of labels in our data

  # Load the MainLayer
  roberta = transformer_model.layers[0]

  # Build the model inputs
  input_ids = layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
  attention_mask = layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')
  token_type_ids = layers.Input(shape=(max_length,), name='token_type_ids', dtype='int32') #original code called this variable token_ids
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids}

  # Load the Transformers BERT model as a layer in a Keras model
  roberta_model = roberta(inputs)[1]
  dropout = layers.Dropout(configuration.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(roberta_model, training=False)

  # Then build the model output
  #emotion = layers.Dense(units=nb_labels, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output) #possibly change activation to softmax
  emotion = layers.Dense(units=nb_labels, activation="softmax", kernel_initializer=TruncatedNormal(stddev=configuration.initializer_range), name='emotion')(pooled_output) #possibly change activation to softmax
  outputs = emotion

  # And combine it all in a model object
  model = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='roBERTa_Label')

  return model

In [48]:
# Creating a model instance
model = create_model(num_labels)

# Take a look at the model
model.summary()

Model: "roBERTa_Label"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]',  

In [49]:
# Creating train, validation and test variables
X_train = ds_train['clean_text']
y_train = ds_train.loc[:, GE_taxonomy].values.astype(float)

X_valid = ds_valid['clean_text']
y_valid = ds_valid.loc[:, GE_taxonomy].values.astype(float)

X_test = ds_test['clean_text']
y_test = ds_test.loc[:, GE_taxonomy].values.astype(float)

In [50]:
# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
valid_token = tokenizer(
    text = X_valid.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [51]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids 
# train = {'input_id': train_token['input_ids'], 'input_2': train_token['attention_mask'],'input_3': train_token['token_type_ids']}
# val = {'input_id': valid_token['input_ids'], 'input_2': valid_token['attention_mask'],'input_3': valid_token['token_type_ids']}
# test = {'input_1': test_token['input_ids'], 'input_2': test_token['attention_mask'],'input_3': test_token['token_type_ids']}

train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_type_ids': train_token['token_type_ids']}
val = {'input_ids': valid_token['input_ids'], 'attention_mask': valid_token['attention_mask'],'token_type_ids': valid_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_type_ids': test_token['token_type_ids']}


In [52]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_valid)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(16)

In [53]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        #compute_class_weight(class_weight, *, classes, y)
        weights[i] = compute_class_weight('balanced', classes = [0.,1.], y = y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)

In [54]:
# Custom loss function for multilabel
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [55]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1.e-06,
    weight_decay=None
    )

# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics=["accuracy"] #try auc and F1 score 
    )

# train the model
history = model.fit(train_tensor, 
                    epochs=8, 
                    validation_data=val_tensor,
                    )

Epoch 1/8
2270/2270 [==============================] - 1103s 457ms/step - loss: 0.6545 - accuracy: 0.1646 - val_loss: 0.5374 - val_accuracy: 0.4703
Epoch 2/8
2270/2270 [==============================] - 1009s 445ms/step - loss: 0.4919 - accuracy: 0.4718 - val_loss: 0.4365 - val_accuracy: 0.5253
Epoch 3/8
2270/2270 [==============================] - 1001s 441ms/step - loss: 0.4224 - accuracy: 0.5268 - val_loss: 0.3930 - val_accuracy: 0.5400
Epoch 4/8
2270/2270 [==============================] - 988s 435ms/step - loss: 0.3832 - accuracy: 0.5475 - val_loss: 0.3662 - val_accuracy: 0.5574
Epoch 5/8
2270/2270 [==============================] - 1010s 445ms/step - loss: 0.3564 - accuracy: 0.5630 - val_loss: 0.3479 - val_accuracy: 0.5627
Epoch 6/8
2270/2270 [==============================] - 992s 437ms/step - loss: 0.3356 - accuracy: 0.5719 - val_loss: 0.3341 - val_accuracy: 0.5719
Epoch 7/8
2270/2270 [==============================] - 943s 416ms/step - loss: 0.3174 - accuracy: 0.5848 - val_los

In [56]:
y_pred_proba = model.predict(test)

144/144 [==============================] - 35s 227ms/step


In [57]:
test

{'input_ids': <tf.Tensor: shape=(4590, 32), dtype=int32, numpy=
 array([[    0, 37145,   261, ...,     1,     1,     1],
        [    0,  2527,    42, ...,     1,     1,     1],
        [    0,  8987,  1994, ...,     1,     1,     1],
        ...,
        [    0,  8987,   266, ...,     1,     1,     1],
        [    0,   118,   657, ...,     1,     1,     1],
        [    0, 11970,   101, ...,     1,     1,     1]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(4590, 32), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(4590, 32), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ...,

In [58]:
y_pred_proba = model.predict(test)

# from probabilities to labels using a given threshold
def proba_to_labels(y_pred_proba, threshold=0.8):
    
    y_pred_labels = np.zeros_like(y_pred_proba)
    
    for i in range(y_pred_proba.shape[0]):
        for j in range(y_pred_proba.shape[1]):
            if y_pred_proba[i][j] > threshold:
                y_pred_labels[i][j] = 1
            else:
                y_pred_labels[i][j] = 0
                
    return y_pred_labels

144/144 [==============================] - 34s 234ms/step


In [59]:
# Generate labels
y_pred_labels = proba_to_labels(y_pred_proba)

In [60]:
# Model evaluation function 
def model_eval(y_true, y_pred_labels, emotions):
    
    # Defining variables
    precision = []
    recall = []
    f1 = []
    
    # Per emotion evaluation      
    idx2emotion = {i: e for i, e in enumerate(emotions)}
    
    for i in range(len(emotions)):
   
        # Computing precision, recall and f1-score
        p, r, f1_score, _ = precision_recall_fscore_support(y_true[:, i], y_pred_labels[:, i], average="binary")
        
        # Append results in lists
        precision.append(round(p, 2))
        recall.append(round(r, 2))
        f1.append(round(f1_score, 2))
    
    # Macro evaluation
    macro_p, macro_r, macro_f1_score, _ = precision_recall_fscore_support(y_true, y_pred_labels, average="macro")
    
    # Append results in lists
    precision.append(round(macro_p, 2))
    recall.append(round(macro_r, 2))
    f1.append(round(macro_f1_score, 2))
    
    # Converting results to a dataframe
    df_results = pd.DataFrame({"Precision":precision, "Recall":recall, 'F1':f1})
    df_results.index = emotions+['MACRO-AVERAGE']
    
    return df_results

In [61]:
model_eval(y_test, y_pred_labels, GE_taxonomy) #for the record, paper had F1 = 0.46

,Precision,Recall,F1
approval,0.83,0.27,0.40
confusion,1.00,0.03,0.06
desire,0.55,0.39,0.46
disapproval,0.73,0.20,0.31
fear,0.69,0.69,0.69
gratitude,0.92,0.88,0.90
joy,0.81,0.61,0.70
love,0.81,0.51,0.63
neutral,0.94,0.02,0.04
optimism,0.74,0.49,0.59
